In [1]:
from dotenv import load_dotenv
import os
import json
import pathlib
import pandas as pd
import datetime

from polygon import RESTClient


load_dotenv()
%load_ext dotenv
%dotenv

# Check reference store

In [2]:
INTERIM_DATA_FOLDER_PATH = pathlib.Path("../data/interim")
RAW_DATA_FOLDER_PATH = pathlib.Path("../data/raw")

In [3]:
def get_reference_to_fill():
    with open(INTERIM_DATA_FOLDER_PATH.joinpath('reference_store.json'), "r") as f:
        reference_store = json.load(f)

    with open(INTERIM_DATA_FOLDER_PATH.joinpath('latest_config.json'), "r") as f:
        last_config = json.load(f)
        
    last_date = datetime.datetime.strptime((last_config['date']), '%Y-%m-%d')
    end_date = datetime.datetime(2023, 12, 31)
    last_etf = last_config['etf']
    listed_etfs = list(reference_store.keys())
    last_company = last_config['company']
    listed_companies = list(reference_store[last_etf].keys())

    reached_last_date = last_date == end_date
    reached_last_company = listed_companies.index(last_company) + 1 == len(listed_companies)
    reached_last_etf = listed_etfs.index(last_etf) + 1 == len(listed_etfs)

    # End of the reference store
    if reached_last_date and reached_last_company and reached_last_etf:
        return reference_store

    for etf in listed_etfs[:listed_etfs.index(last_etf)]:
        reference_store.pop(etf)

    for company in listed_companies[: listed_companies.index(last_company)]:
        reference_store[last_etf].pop(company)

    for date in list(reference_store[last_etf][last_company].keys()):
        reference_store[last_etf][last_company].pop(date)
        if datetime.datetime.strptime(date, '%Y-%m-%d') == last_date:
            break
        
    return reference_store

In [4]:
with open(INTERIM_DATA_FOLDER_PATH.joinpath('reference_store.json'), "r") as f:
    reference_store = json.load(f)

with open(INTERIM_DATA_FOLDER_PATH.joinpath('latest_config.json'), "r") as f:
    last_config = json.load(f)

In [5]:
aber = get_reference_to_fill()

In [6]:
aber['VIS']['CAT'].update({
    '2023-03-10': [{'a': 1, 'b': 2}, {'a': 3, 'b': 4}],
    '2023-03-11': [{'a': 1, 'b': 2}, {'a': 3, 'b': 4}],
    '2023-03-12': [{'a': 1, 'b': 2}, {'a': 3, 'b': 4}],
})

In [7]:
for etf, company_dict in aber.items():
    for company, news_dict in company_dict.items():
        reference_store[etf][company].update(news_dict)

# Polygon

In [ ]:
POLYGON_API_KEY = os.environ["POLYGON_API_KEY"]

In [ ]:
client = RESTClient(api_key=POLYGON_API_KEY)

In [ ]:
response = client.list_ticker_news(ticker='C', published_utc='2024-02-01')
aber = [a for a in response]

In [ ]:
aber

In [ ]:
for obj in aber:
    print(obj.title)